# 🎯 LangGraph Aplicado a Codificación Automática

Este notebook muestra cómo **refactorizar tu sistema de codificación** usando LangGraph.

## 📋 Lo que vamos a hacer:

1. **Analizar** tu flujo actual
2. **Diseñar** el grafo de estados
3. **Implementar** nodos clave
4. **Probar** con datos reales
5. **Comparar** con tu solución actual

---


## 🔧 Setup


In [4]:
import os
import sys
from pathlib import Path

# Agregar ruta del proyecto
project_root = Path.cwd().parent
sys.path.append(str(project_root / "backend" / "src"))

from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Verificar
assert os.getenv("OPENAI_API_KEY"), "❌ Falta OPENAI_API_KEY"
print("✅ Setup completo")


✅ Setup completo


In [5]:
# Imports necesarios
from typing import TypedDict, List, Dict, Literal
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import pandas as pd

print("✅ Imports cargados")


✅ Imports cargados


---

## 📊 Tu Flujo Actual

```
Cargar Datos
    ↓
Por cada pregunta:
    ↓
    Dividir en batches (20 respuestas)
    ↓
    Por cada batch:
        ↓
        Llamar GPT con prompt híbrido
        ↓
        Parsear respuesta
        ↓
        Acumular códigos nuevos
    ↓
    Normalizar códigos nuevos
    ↓
    Exportar a Excel
```

---


## 🎨 Diseño del Grafo LangGraph

```mermaid
graph TD
    START --> cargar[Cargar Datos]
    cargar --> preparar[Preparar Batch]
    preparar --> codificar[Codificar Batch]
    codificar --> acumular[Acumular Resultados]
    acumular --> decision{¿Más batches?}
    decision -->|Sí| preparar
    decision -->|No| normalizar[Normalizar Códigos]
    normalizar --> exportar[Exportar Excel]
    exportar --> END
```

**Ventajas:**
- ✅ Cada nodo es testeable por separado
- ✅ Fácil agregar pasos (ej: validación, retry)
- ✅ Progreso rastreable nodo por nodo
- ✅ Checkpointing automático

---


## 🏗️ Definir el Estado


In [6]:
class EstadoCodificacion(TypedDict):
    """Estado que fluye por el grafo"""
    
    # Configuración
    pregunta: str
    modelo_gpt: str
    batch_size: int
    
    # Datos de entrada
    respuestas: List[Dict]  # Lista de {texto, fila_excel}
    catalogo: List[Dict]     # Catálogo histórico
    
    # Control de flujo
    batch_actual: int
    total_batches: int
    batch_respuestas: List[Dict]  # Respuestas del batch actual
    
    # Resultados acumulados
    codificaciones: List[Dict]  # Todas las codificaciones
    codigos_nuevos_global: Dict[str, int]  # {descripcion: codigo_num}
    contador_codigo_nuevo: int
    
    # Métricas
    costo_total: float
    tokens_total: int
    
    # Progreso
    mensaje_estado: str
    progreso_pct: float

# Ejemplo de estado inicial
estado_ejemplo = {
    "pregunta": "¿Qué te gusta del producto?",
    "modelo_gpt": "gpt-4o-mini",
    "batch_size": 20,
    "respuestas": [],
    "catalogo": [],
    "batch_actual": 0,
    "total_batches": 0,
    "batch_respuestas": [],
    "codificaciones": [],
    "codigos_nuevos_global": {},
    "contador_codigo_nuevo": 1,
    "costo_total": 0.0,
    "tokens_total": 0,
    "mensaje_estado": "Iniciando...",
    "progreso_pct": 0.0
}

print("✅ Estado definido")
print(f"\n📊 Campos del estado: {list(estado_ejemplo.keys())}")


✅ Estado definido

📊 Campos del estado: ['pregunta', 'modelo_gpt', 'batch_size', 'respuestas', 'catalogo', 'batch_actual', 'total_batches', 'batch_respuestas', 'codificaciones', 'codigos_nuevos_global', 'contador_codigo_nuevo', 'costo_total', 'tokens_total', 'mensaje_estado', 'progreso_pct']


---

## 🔨 Implementar Nodos

### Nodo 1: Cargar Datos


In [ ]:
def nodo_cargar_datos(state: EstadoCodificacion) -> EstadoCodificacion:
    """
    Simula carga de datos (en producción leerías de Excel)
    """
    # Datos de ejemplo
    respuestas_ejemplo = [
        {"texto": "Me gusta el sabor", "fila_excel": 1},
        {"texto": "Es muy versátil para cocinar", "fila_excel": 2},
        {"texto": "Tiene buen precio y calidad", "fila_excel": 3},
        {"texto": "El sabor es excelente", "fila_excel": 4},
        {"texto": "Lo uso en muchas comidas diferentes", "fila_excel": 5},
    ]
    
    catalogo_ejemplo = [
        {"codigo": 1, "descripcion": "Sabor agradable"},
        {"codigo": 2, "descripcion": "Buen precio"},
        {"codigo": 3, "descripcion": "Calidad del producto"},
    ]
    
    total_batches = (len(respuestas_ejemplo) + state["batch_size"] - 1) // state["batch_size"]
    
    print(f" Cargados: {len(respuestas_ejemplo)} respuestas")
    print(f" Catálogo: {len(catalogo_ejemplo)} códigos")
    print(f" Total batches: {total_batches}")
    
    return {
        **state,
        "respuestas": respuestas_ejemplo,
        "catalogo": catalogo_ejemplo,
        "total_batches": total_batches,
        "mensaje_estado": f"Datos cargados: {len(respuestas_ejemplo)} respuestas"
    }

# Probar nodo
print("\n Probando nodo_cargar_datos:")
resultado = nodo_cargar_datos(estado_ejemplo)
print(f"Estado actualizado: total_batches={resultado['total_batches']}")



🧪 Probando nodo_cargar_datos:
📂 Cargados: 5 respuestas
📚 Catálogo: 3 códigos
📦 Total batches: 1
Estado actualizado: total_batches=1


### Nodo 2: Preparar Batch


In [8]:
def nodo_preparar_batch(state: EstadoCodificacion) -> EstadoCodificacion:
    """
    Extrae el siguiente batch de respuestas
    """
    inicio = state["batch_actual"] * state["batch_size"]
    fin = min(inicio + state["batch_size"], len(state["respuestas"]))
    
    batch = state["respuestas"][inicio:fin]
    
    progreso = ((state["batch_actual"] + 1) / state["total_batches"]) * 100
    
    print(f"\n📦 Batch {state['batch_actual'] + 1}/{state['total_batches']}")
    print(f"   Respuestas {inicio+1} a {fin}")
    print(f"   Progreso: {progreso:.1f}%")
    
    return {
        **state,
        "batch_respuestas": batch,
        "progreso_pct": progreso,
        "mensaje_estado": f"Procesando batch {state['batch_actual'] + 1}/{state['total_batches']}"
    }

# Probar
print("\n🧪 Probando nodo_preparar_batch:")
estado_test = {**resultado, "batch_actual": 0}
resultado2 = nodo_preparar_batch(estado_test)
print(f"Batch extraído: {len(resultado2['batch_respuestas'])} respuestas")



🧪 Probando nodo_preparar_batch:

📦 Batch 1/1
   Respuestas 1 a 5
   Progreso: 100.0%
Batch extraído: 5 respuestas


### Nodo 3: Codificar Batch (con GPT)


In [9]:
# Definir esquemas Pydantic para respuesta estructurada
class ResultadoCodificacionGPT(BaseModel):
    respuesta_id: int = Field(description="ID de la respuesta (1-based)")
    decision: Literal["historico", "nuevo", "mixto", "rechazar"] = Field(
        description="Tipo de codificación"
    )
    codigos_historicos: List[int] = Field(
        default_factory=list,
        description="Códigos del catálogo histórico"
    )
    codigos_nuevos: List[str] = Field(
        default_factory=list,
        description="Descripciones de códigos nuevos"
    )
    justificacion: str = Field(description="Por qué se asignaron estos códigos")

class RespuestaBatchGPT(BaseModel):
    codificaciones: List[ResultadoCodificacionGPT] = Field(
        description="Lista de codificaciones para el batch"
    )

print("✅ Esquemas Pydantic definidos")


✅ Esquemas Pydantic definidos


In [10]:
def nodo_codificar_batch(state: EstadoCodificacion) -> EstadoCodificacion:
    """
    Llama a GPT para codificar el batch actual
    """
    # Preparar prompt
    catalogo_str = "\n".join([
        f"- Código {c['codigo']}: {c['descripcion']}"
        for c in state["catalogo"]
    ])
    
    respuestas_str = "\n".join([
        f"{i+1}. {r['texto']}"
        for i, r in enumerate(state["batch_respuestas"])
    ])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Eres un experto en codificación de respuestas abiertas.

CATÁLOGO HISTÓRICO:
{catalogo}

INSTRUCCIONES:
1. Para cada respuesta, decide:
   - "historico": Si hay códigos del catálogo que apliquen
   - "nuevo": Si se necesitan códigos nuevos (ideas no en catálogo)
   - "mixto": Combinación de históricos y nuevos
   - "rechazar": Si la respuesta no es válida

2. Puedes asignar MÚLTIPLES códigos si hay varias ideas.

3. Para códigos nuevos, sé específico pero no redundante:
   ✅ "Versatilidad de uso"
   ❌ "Versatilidad de uso en comidas" (muy específico)
   ❌ "Versatilidad" (muy genérico)

Responde en formato JSON estructurado."""),
        ("user", "PREGUNTA: {pregunta}\n\nRESPUESTAS:\n{respuestas}")
    ])
    
    # Llamar a GPT
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(RespuestaBatchGPT)
    
    print(f"\n🤖 Llamando a {state['modelo_gpt']}...")
    
    resultado = chain.invoke({
        "catalogo": catalogo_str,
        "pregunta": state["pregunta"],
        "respuestas": respuestas_str
    })
    
    print(f"✅ GPT respondió con {len(resultado.codificaciones)} codificaciones")
    
    # Mostrar algunas
    for cod in resultado.codificaciones[:2]:
        print(f"   - Respuesta {cod.respuesta_id}: {cod.decision}")
        if cod.codigos_historicos:
            print(f"     Históricos: {cod.codigos_historicos}")
        if cod.codigos_nuevos:
            print(f"     Nuevos: {cod.codigos_nuevos}")
    
    # Convertir a formato interno
    codificaciones_batch = []
    for cod in resultado.codificaciones:
        codificaciones_batch.append({
            "fila_excel": state["batch_respuestas"][cod.respuesta_id - 1]["fila_excel"],
            "texto": state["batch_respuestas"][cod.respuesta_id - 1]["texto"],
            "decision": cod.decision,
            "codigos_historicos": cod.codigos_historicos,
            "codigos_nuevos": cod.codigos_nuevos,
            "justificacion": cod.justificacion
        })
    
    return {
        **state,
        "codificaciones": state["codificaciones"] + codificaciones_batch,
        "mensaje_estado": f"Batch {state['batch_actual'] + 1} codificado"
    }

# NO probamos aún (consume API)


### Nodo 4: Incrementar Batch


In [11]:
def nodo_incrementar_batch(state: EstadoCodificacion) -> EstadoCodificacion:
    """
    Incrementa el contador de batch
    """
    nuevo_batch = state["batch_actual"] + 1
    
    print(f"➡️  Avanzando a batch {nuevo_batch + 1}")
    
    return {
        **state,
        "batch_actual": nuevo_batch
    }


### Nodo 5: Normalizar Códigos


In [12]:
def similitud_jaccard(desc1: str, desc2: str) -> float:
    """Calcula similitud entre descripciones"""
    palabras1 = set(desc1.lower().split())
    palabras2 = set(desc2.lower().split())
    
    interseccion = len(palabras1 & palabras2)
    union = len(palabras1 | palabras2)
    
    return interseccion / union if union > 0 else 0.0

def nodo_normalizar(state: EstadoCodificacion) -> EstadoCodificacion:
    """
    Unifica códigos nuevos redundantes
    """
    print("\n🔧 Normalizando códigos nuevos...")
    
    # Recolectar todas las descripciones nuevas
    descripciones_vistas = {}
    mapeo = {}  # {descripcion_original: descripcion_normalizada}
    contador = 1
    
    for cod in state["codificaciones"]:
        for desc in cod["codigos_nuevos"]:
            # Buscar si ya existe similar
            encontrado = False
            for desc_norm, _ in descripciones_vistas.items():
                if similitud_jaccard(desc, desc_norm) > 0.6:
                    mapeo[desc] = desc_norm
                    encontrado = True
                    print(f"   ♻️  '{desc}' → '{desc_norm}' (similitud)")
                    break
            
            if not encontrado:
                mapeo[desc] = desc
                descripciones_vistas[desc] = contador
                contador += 1
    
    # Aplicar normalización
    codigos_nuevos_global = {}
    contador_nuevo = 1
    
    for desc_norm in descripciones_vistas.keys():
        codigo_final = f"N{contador_nuevo}"
        codigos_nuevos_global[desc_norm] = codigo_final
        contador_nuevo += 1
    
    print(f"✅ {len(codigos_nuevos_global)} códigos nuevos únicos")
    
    return {
        **state,
        "codigos_nuevos_global": codigos_nuevos_global,
        "contador_codigo_nuevo": contador_nuevo,
        "mensaje_estado": "Códigos normalizados"
    }


### Nodo 6: Exportar Resultados


In [13]:
def nodo_exportar(state: EstadoCodificacion) -> EstadoCodificacion:
    """
    Prepara DataFrame para exportar
    """
    print("\n💾 Preparando exportación...")
    
    filas = []
    for cod in state["codificaciones"]:
        # Combinar códigos
        codigos_finales = []
        
        # Históricos
        if cod["codigos_historicos"]:
            codigos_finales.extend([str(c) for c in cod["codigos_historicos"]])
        
        # Nuevos
        if cod["codigos_nuevos"]:
            for desc in cod["codigos_nuevos"]:
                desc_norm = desc  # Ya normalizado en paso anterior
                codigo_nuevo = state["codigos_nuevos_global"].get(desc_norm, "?")
                codigos_finales.append(codigo_nuevo)
        
        filas.append({
            "fila": cod["fila_excel"],
            "respuesta": cod["texto"],
            "decision": cod["decision"],
            "codigos": " | ".join(codigos_finales) if codigos_finales else "SIN_CODIGO",
            "justificacion": cod["justificacion"]
        })
    
    df = pd.DataFrame(filas)
    
    print(f"✅ DataFrame creado: {len(df)} filas")
    print(f"\n📊 Primeras filas:")
    print(df.head())
    
    return {
        **state,
        "mensaje_estado": "Exportación completa",
        "progreso_pct": 100.0
    }


In [14]:
# Función de decisión: ¿continuar o terminar?
def decidir_continuar(state: EstadoCodificacion) -> str:
    if state["batch_actual"] < state["total_batches"]:
        return "preparar_batch"
    else:
        return "normalizar"

# Construir grafo
workflow = StateGraph(EstadoCodificacion)

# Agregar nodos
workflow.add_node("cargar", nodo_cargar_datos)
workflow.add_node("preparar_batch", nodo_preparar_batch)
workflow.add_node("codificar", nodo_codificar_batch)
workflow.add_node("incrementar", nodo_incrementar_batch)
workflow.add_node("normalizar", nodo_normalizar)
workflow.add_node("exportar", nodo_exportar)

# Definir flujo
workflow.set_entry_point("cargar")
workflow.add_edge("cargar", "preparar_batch")
workflow.add_edge("preparar_batch", "codificar")
workflow.add_edge("codificar", "incrementar")

# Decisión: ¿más batches?
workflow.add_conditional_edges(
    "incrementar",
    decidir_continuar,
    {
        "preparar_batch": "preparar_batch",
        "normalizar": "normalizar"
    }
)

workflow.add_edge("normalizar", "exportar")
workflow.add_edge("exportar", END)

# Compilar
app = workflow.compile()

print("✅ Grafo de codificación compilado")


✅ Grafo de codificación compilado


---

## 🧪 Ejecutar el Grafo (ADVERTENCIA: Consume API)

⚠️ **Esta celda llamará a OpenAI GPT** - descomenta para ejecutar:


In [16]:
# Descomenta para ejecutar:
estado_inicial = {
    "pregunta": "¿Qué te gusta del producto?",
    "modelo_gpt": "gpt-4o-mini",
    "batch_size": 3,  # Pequeño para prueba
    "respuestas": [],
    "catalogo": [],
    "batch_actual": 0,
    "total_batches": 0,
    "batch_respuestas": [],
    "codificaciones": [],
    "codigos_nuevos_global": {},
    "contador_codigo_nuevo": 1,
    "costo_total": 0.0,
    "tokens_total": 0,
    "mensaje_estado": "Iniciando...",
    "progreso_pct": 0.0
}

print("🚀 Ejecutando grafo completo...\n")
print("="*60)

resultado_final = app.invoke(estado_inicial)

print("\\n" + "="*60)
print("🎉 ¡COMPLETADO!")
print("="*60)
print(f"✅ {len(resultado_final['codificaciones'])} respuestas codificadas")
print(f"✅ {len(resultado_final['codigos_nuevos_global'])} códigos nuevos")


print("⚠️ Celda comentada - descomenta para ejecutar")


🚀 Ejecutando grafo completo...

📂 Cargados: 5 respuestas
📚 Catálogo: 3 códigos
📦 Total batches: 2

📦 Batch 1/2
   Respuestas 1 a 3
   Progreso: 50.0%

🤖 Llamando a gpt-4o-mini...
✅ GPT respondió con 3 codificaciones
   - Respuesta 1: historico
     Históricos: [1]
   - Respuesta 2: nuevo
     Nuevos: ['Versatilidad de uso']
➡️  Avanzando a batch 2

📦 Batch 2/2
   Respuestas 4 a 5
   Progreso: 100.0%

🤖 Llamando a gpt-4o-mini...
✅ GPT respondió con 2 codificaciones
   - Respuesta 1: historico
     Históricos: [1]
   - Respuesta 2: nuevo
     Nuevos: ['Versatilidad de uso']
➡️  Avanzando a batch 3

🔧 Normalizando códigos nuevos...
   ♻️  'Versatilidad de uso' → 'Versatilidad de uso' (similitud)
✅ 1 códigos nuevos únicos

💾 Preparando exportación...
✅ DataFrame creado: 5 filas

📊 Primeras filas:
   fila                            respuesta   decision codigos  \
0     1                    Me gusta el sabor  historico       1   
1     2         Es muy versátil para cocinar      nuevo      N

---

## 🎯 Ventajas de LangGraph vs. Tu Código Actual

| Aspecto | Código Actual | Con LangGraph |
|---------|--------------|---------------|
| **Organización** | Todo en `codificador.py` | Cada nodo = función independiente |
| **Testing** | Difícil testear pasos intermedios | Cada nodo se prueba por separado |
| **Progreso** | Callback manual | Nativo en cada transición |
| **Reintentos** | Implementar manualmente | Retry automático por nodo |
| **Checkpointing** | No hay | Guardar/recuperar estado |
| **Observabilidad** | Logs básicos | Trazas completas (LangSmith) |
| **Modificación** | Tocar lógica existente | Agregar/quitar nodos |
| **Debugging** | Print statements | Ver estado en cada paso |

---


## 🚀 Próximos Pasos

### 1. **Mejorar este notebook:**
   - Agregar nodo de validación (detectar respuestas inválidas)
   - Implementar retry logic si GPT falla
   - Agregar cálculo de métricas (costos, tokens)

### 2. **Migrar a producción:**
   - Mover nodos a `backend/src/cod_backend/core/langgraph_nodes.py`
   - Crear grafo en `backend/src/cod_backend/core/langgraph_workflow.py`
   - Actualizar API para usar el grafo

### 3. **Agregar funcionalidades avanzadas:**
   - **Streaming**: Enviar progreso en tiempo real al frontend (SSE)
   - **Checkpointing**: Guardar progreso y recuperar si falla
   - **Observabilidad**: Integrar LangSmith para trazas
   - **Multi-agente**: Un agente clasifica, otro codifica

---

## ➡️ Siguiente Notebook

Continúa con **`03_langgraph_streaming.ipynb`** para aprender a enviar progreso en tiempo real al frontend.
